In [144]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.ensemble import IsolationForest

from sklearn.svm import OneClassSVM
import joblib
from joblib import dump
from keras.models import model_from_json

In [145]:
filePath = "C:/Users/raul.blanco/Documents/4 - Personales/UBA/Repositorios/IA/Trabajo-Final/solar_wind.csv"
dF = pd.read_csv(filepath_or_buffer=filePath, header=0, sep=",")

df_A = dF.loc[dF['period']=='train_a']
df_A = df_A.dropna()

In [146]:
train_data = df_A.sample(frac=0.8, random_state=0)
test_data = df_A.drop(train_data.index)

In [147]:
features = ['bt']
anomaly_factor = 0.17
# Crear una copia del DataFrame para no modificar el original
df_new = train_data.copy()

for a in features:
  num_replace = int(train_data[a].size * anomaly_factor)

  # Obtener una muestra aleatoria de índices de fila para reemplazar
  replace_idx = np.random.choice(train_data.index, size=num_replace, replace=False)

  # Reemplazar los valores seleccionados con valores aleatorios entre -90 y 1050
  #new_values = np.random.uniform(-90, 1050, size=num_replace)
  new_values = np.random.uniform(0, 75, size=num_replace)
  df_new.loc[replace_idx, a] = new_values

  # Crear la columna "class" con los valores correspondientes
  df_new.loc[train_data[a] != df_new[a], 'class'] = 1
  df_new.loc[train_data[a] == df_new[a], 'class'] = 0
  train_data = df_new

In [148]:
features = ['bt']
anomaly_factor = 0.012
# Crear una copia del DataFrame para no modificar el original
df_new_test = test_data.copy()

for a in features:
  num_replace = int(test_data[a].size * anomaly_factor)

  # Obtener una muestra aleatoria de índices de fila para reemplazar
  replace_idx = np.random.choice(test_data.index, size=num_replace, replace=False)

  # Reemplazar los valores seleccionados con valores aleatorios entre -90 y 1050
  new_values = np.random.uniform(0, 75, size=num_replace)
  df_new_test.loc[replace_idx, a] = new_values

  # Crear la columna "class" con los valores correspondientes
  df_new_test.loc[test_data[a] != df_new_test[a], 'class'] = 1
  df_new_test.loc[test_data[a] == df_new_test[a], 'class'] = 0
  test_data = df_new_test

In [149]:
train_timedelta = train_data[['timedelta']]
test_timedelta = test_data[['timedelta']]

y = test_data['class']
y_full = test_data
train_data = train_data.drop(["timedelta", "period", "source", 'class'], axis=1)
test_data = test_data.drop(["timedelta", "period", "source", 'class'], axis=1)
#y_full = y_full.to_numpy() #esto lo tengo que cargar en el archivo test

In [150]:
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [151]:
test_data_pd = pd.DataFrame(test_data, columns=['bx_gse', 'by_gse', 'bz_gse', 'theta_gse',
       'phi_gse', 'bx_gsm', 'by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm', 'bt',
       'density', 'speed', 'temperature'])
test_data_pd.reset_index()

,index,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm,bt,density,speed,temperature
0,0,-0.791682,0.687973,0.202381,0.204546,-0.495391,-0.791645,0.702823,0.204953,0.204883,-0.492878,-0.369382,-0.605288,-0.508709,-0.233013
1,1,-0.773036,0.687973,0.158560,0.159756,-0.500518,-0.772999,0.702823,0.161572,0.160708,-0.497996,-0.375426,-0.646606,-0.560023,-0.115806
2,2,-1.048070,0.283227,0.216219,0.211482,-0.275259,-1.048025,0.294834,0.218652,0.211723,-0.273135,-0.358639,-0.741637,-0.905782,-0.438237
3,3,-1.022431,0.195147,0.435323,0.441213,-0.237431,-1.022387,0.206049,0.435555,0.438297,-0.235373,-0.359982,-0.578432,-0.677039,0.383221
4,4,-1.006116,0.453094,0.218525,0.211771,-0.355131,-1.006072,0.466063,0.220935,0.212008,-0.352865,-0.352595,-0.873853,-0.885468,-0.627182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278906,278906,-0.467701,0.459386,-0.429561,-0.669299,-0.499948,-0.467675,0.584443,-0.201455,-0.330345,-0.555548,-0.482864,0.084717,-0.998359,-0.661721
278907,278907,-0.344168,0.518105,-0.438787,-0.716112,-0.579364,-0.344146,0.641519,-0.190039,-0.329775,-0.631298,-0.497637,0.173550,-1.038986,-0.602403
278908,278908,-0.460708,0.591505,-0.374208,-0.555445,-0.570135,-0.460682,0.692254,-0.105561,-0.182715,-0.606958,-0.466748,0.107442,-1.034331,-0.665081
278909,278909,-0.423416,0.597796,-0.302711,-0.471644,-0.586770,-0.423391,0.677456,-0.039349,-0.094366,-0.614124,-0.480178,0.123969,-1.044382,-0.599593


In [152]:
test_timedelta.reset_index(drop=True, inplace = True)
test_data_out = pd.concat([test_timedelta, test_data_pd], axis = 'columns')

In [154]:
train_data_LSTM = train_data.reshape(train_data.shape[0],1,train_data.shape[1])
#test_data_LSTM = test_data.reshape(test_data.shape[0],1,test_data.shape[1])

#Modelo
input_dim = train_data_LSTM.shape[2]
timesteps = 1
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(64, activation='relu', return_sequences=True)(inputs)
encoded = LSTM(32, activation='relu', return_sequences=True)(inputs)
encoded = LSTM(16, activation='relu', return_sequences=False)(encoded)
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(16, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(64, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(input_dim, activation='linear', return_sequences=True)(decoded)
model_autoencoder = Model(inputs, decoded)
model_autoencoder.compile(optimizer='adam', loss='mse')

#Entreno el modelo
model_autoencoder.fit(train_data_LSTM, train_data_LSTM, epochs=1, batch_size=320, validation_split=0.1)
#Guardo el modelo.
#joblib.dump(model_autoencoder, 'model_autoencoder.joblib')

# serializa el modelo para JSON
model_json = model_autoencoder.to_json()
with open("model_LSTM.json", "w") as json_file:
  json_file.write(model_json)
#serializan los pesos (weights) para HDF5
model_autoencoder.save_weights("model_LSTM.h5")
print("Modelo guardado en el PC")

3138/3138 [==============================] - 27s 7ms/step - loss: 0.1610 - val_loss: 0.0219
Modelo guardado en el PC


In [ ]:
test_data_out.to_csv('test_data.csv')
y_full.to_csv('y_full.csv')
print("Se guardó dataset de test con el nombre test_data_LSTM.csv")

In [ ]:
y_full.shape

In [ ]:
test_data_out.shape